# PyTorch - Neural Networks Tutorial

# Import required libraries

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torchsummary import summary
%matplotlib inline

# Neural Networks

Neural networks can be constructed using the ``torch.nn`` package. ``nn`` depends on ``autograd`` to define models and differentiate them. An ``nn.Module`` contains layers, and a method ``forward(input)`` that returns the ``output``.

A typical training procedure for a neural network is as follows:

- Define the neural network that has some learnable parameters (or weights)
- Iterate over a dataset of inputs
- Process input through the network
- Compute the loss (how far is the output from being correct)
- Propagate gradients back into the network’s parameters
- Update the weights of the network, typically using a simple update rule:
  ``weight = weight - learning_rate * gradient``.

## Define the network

Let’s first define this network:

In [2]:
class NN(nn.Module):

    def __init__(self):
        super(NN, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 5) # Single-channel input, 6 o/p channels, 5 x 5 kernel
        self.conv2 = nn.Conv2d(6, 16, 5) # i/p channels = 6, o/p channels = 16, 5 x 5 kernel
        self.fc1 = nn.Linear(16 * 5 * 5, 120)  # i/p channels = 400, o/p channels = 120
        self.fc2 = nn.Linear(120, 84)  # i/p channels = 120, o/p channels = 84
        self.fc3 = nn.Linear(84, 10)  # i/p channels = 84, o/p channels = 10

    def forward(self, x):
        # Conv1 -> ReLU -> MaxPool
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # Conv2 -> ReLU -> MaxPool        
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = torch.flatten(x, 1) # flatten all dimensions except the batch dimension
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


model = NN()
print(model)

NN(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [3]:
summary(model, input_size = (1, 32, 32))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 6, 28, 28]             156
            Conv2d-2           [-1, 16, 10, 10]           2,416
            Linear-3                  [-1, 120]          48,120
            Linear-4                   [-1, 84]          10,164
            Linear-5                   [-1, 10]             850
Total params: 61,706
Trainable params: 61,706
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.05
Params size (MB): 0.24
Estimated Total Size (MB): 0.29
----------------------------------------------------------------


## Network parameters

<b>The learnable parameters of a model are returned by ``net.parameters()``.</b>

In [4]:
params = list(model.parameters())
print(len(params))
print(params[0].size())  # conv1's .weight

10
torch.Size([6, 1, 5, 5])


## Forward-prop a random input through the modelm

In [5]:
# Apply a random 32 x 32 input
input = torch.randn(1, 1, 32, 32)
out = model(input)
print(out.shape)
print(out)

torch.Size([1, 10])
tensor([[ 0.0682,  0.0391, -0.0772, -0.0445,  0.0407,  0.1110,  0.0845, -0.0351,
          0.0278,  0.0175]], grad_fn=<AddmmBackward>)


Zero the gradient buffers of all parameters and backprops with random
gradients:



<b>Note:</b> ``torch.nn`` only supports mini-batches. The entire ``torch.nn`` package only supports inputs that are a mini-batch of samples, and not a single sample.

For example, ``nn.Conv2d`` will take in a 4D Tensor of ``nSamples x nChannels x Height x Width``. If we have only a single sample, use ``input.unsqueeze(0)`` to add a fake batch dimension.

## Loss Function

A loss function takes the (output, target) pair of inputs, and computes a value that estimates how far away the output is from the target.

There are several different [`loss functions`](https://pytorch.org/docs/nn.html#loss-functions) under the nn package. A simple loss is: ``nn.MSELoss`` which computes the mean-squared error between the input and the target.

In [6]:
output = model(input)
print(output.shape)

torch.Size([1, 10])


In [7]:
target = torch.randn(10)  # a dummy target, for example
print(target.shape)
target = target.view(1, -1)  # make it the same shape as output
print(target.shape)

torch.Size([10])
torch.Size([1, 10])


In [8]:
criterion = nn.MSELoss()
loss = criterion(output, target)
print(f"Computed loss is {loss:0.4f}")
loss_calc = torch.mean((output - target) ** 2)
print(f"Calculated loss is {loss_calc:0.4f}")

Computed loss is 0.3802
Calculated loss is 0.3802


If we follow ``loss`` in the backward direction, using its ``.grad_fn`` attribute, we will see a graph of computations that looks like this:

::

    input -> conv2d -> relu -> maxpool2d -> conv2d -> relu -> maxpool2d
          -> flatten -> linear -> relu -> linear -> relu -> linear
          -> MSELoss
          -> loss

So, when we call ``loss.backward()``, the whole graph is differentiated w.r.t. the neural net parameters, and all Tensors in the graph that have ``requires_grad=True`` will have their ``.grad`` Tensor accumulated with the gradient.

For illustration, let us follow a few steps backward:

In [9]:
print(loss.grad_fn)  # MSELoss
print(loss.grad_fn.next_functions[0][0])  # Linear
print(loss.grad_fn.next_functions[0][0].next_functions[0][0])  # ReLU

Backprop
--------
To backpropagate the error all we have to do is to ``loss.backward()``. We need to clear the existing gradients though, else gradients will be accumulated to existing gradients.

Now, let's call ``loss.backward()``, and have a look at conv1's bias gradients before and after the backward.

In [10]:
model.zero_grad()     # zeroes the gradient buffers of all parameters

print('conv1.bias.grad before backward')
print(model.conv1.bias.grad)

loss.backward()

print('conv1.bias.grad after backward')
print(model.conv1.bias.grad)

conv1.bias.grad before backward
None
conv1.bias.grad after backward
tensor([-0.0096,  0.0027,  0.0017, -0.0026, -0.0047,  0.0053])


The neural network package contains various modules and loss functions that form the building blocks of deep neural networks. A full list with documentation is available [`here`](https://pytorch.org/docs/nn).

## Update the weights

The simplest update rule used in practice is the Stochastic Gradient Descent (SGD):

     ``weight = weight - learning_rate * gradient``

We can implement this using simple Python code:

.. code:: python

    learning_rate = 0.01
    for f in net.parameters():
        f.data.sub_(f.grad.data * learning_rate)

However, as we use neural networks, we want to use various different update rules such as SGD, Nesterov-SGD, Adam, RMSProp, etc. To enable this, the ``torch.optim`` package can be used that
implements all these methods.

In [11]:
# create the optimizer
optimizer = optim.SGD(model.parameters(), lr = 0.01)

# in the training loop:
optimizer.zero_grad() # zero the gradient buffers
output = model(input) # Forward-prop the input
loss = criterion(output, target) # Define MSE Loss function
loss.backward() # Compute gradients
optimizer.step() # Perform one step of gradient descent